In [1]:
import os
from tqdm import tqdm
from detector import load_model_detector
import pandas as pd
import cv2

ModuleNotFoundError: No module named 'app'

In [20]:
test_name2id = {'klikun':2, 'maliy': 1, 'shipun':3}

In [21]:
# df = pd.read_csv("../create_markup/test.csv")
df = pd.read_csv("../create_markup/test.csv")

In [22]:
path_detector_onnx = r"./model_data/best.onnx"

class_names = ['klikun', 'maliy', 'shipun']
# providers = ['CUDAExecutionProvider', 'CPUExecutionProvider']
providers = ['CPUExecutionProvider']
detector_model = load_model_detector(
    model_path=path_detector_onnx,
    class_names=class_names,
    providers=providers,
    input_shape=(640, 640),
    score_thresh=0.4
)

In [23]:
predict_list = []
root_ds = r"C:\workspace\hakaton\dataset"
for path_img in tqdm(df.path_img.unique()):
    type_name_frame = list(df[df.path_img == path_img]['type_name'].unique())
    if len(type_name_frame) > 1:
        print('ошибка разметки!', path_img)
        continue
    true_label = type_name_frame[0]
    file_path = os.path.join(root_ds, path_img)
    if not os.path.exists(file_path):
        print('not exists', file_path)
        continue
    img = cv2.imread(file_path)[:,:,::-1]
    input_size = (640,640)
    ratio = min(input_size[0] / img.shape[0], input_size[1] / img.shape[1])
    img = cv2.resize(
        img,
        (int(img.shape[1] * ratio), int(img.shape[0] * ratio)),
        interpolation=cv2.INTER_LINEAR)
    result = detector_model(img)
    for row in result:
        predict_list.append([*row, path_img, true_label])
predict_df = pd.DataFrame(predict_list, columns=['name', 'conf', 'x1', 'y1', 'x2', 'y2', 'path_img', 'true_label'])

100%|██████████| 904/904 [04:40<00:00,  3.22it/s]


In [24]:
# predict_df.to_csv('predict_train.csv', index=False)

In [25]:
predict_df.columns

Index(['name', 'conf', 'x1', 'y1', 'x2', 'y2', 'path_img', 'true_label'], dtype='object')

In [26]:

# Отбор name с максимальным coef внутри каждой группы path_img
result = predict_df.loc[predict_df.groupby('path_img')['conf'].idxmax()]
len(result)


901

In [27]:
mask = predict_df.name == predict_df.true_label
len(predict_df[mask])/len(predict_df)
# print(len(predict_df))

0.9643122676579926

In [12]:
# acc = 0
# for path in df.path_img.unique():
#     res = predict_df[predict_df.path_img == path]
#     res = res.groupby('name').count()
#     res.sort_values('conf', ascending=False, inplace=True)
#     # print(res.index[0])
#     if len(res) and res.index[0] == df[df.path_img==path]['type_name'].iloc[0]:
#         acc += 1
#
# acc/len(df.path_img.unique())
# print('Максимальное количество.')


In [13]:
import numpy

In [14]:
acc = 0
for path in tqdm(df.path_img.unique()):
    res = predict_df[predict_df.path_img == path]
    temp = res.groupby('name').count()
    temp = temp.sort_values('conf', ascending=False)
    cls_type = None
    if len(temp)>=2 and temp.conf.iloc[0] == temp.conf.iloc[1]:
        # если нет доминантного класса по количеству
        # самый уверенный бокс
        # res = predict_df[predict_df.path_img == path]
        res = res.sort_values('conf', ascending=False)
        cls_type = res['name'].iloc[0]
    elif len(temp):
        cls_type = temp.index[0]
    if cls_type == df[df.path_img==path]['type_name'].iloc[0]:
        acc += 1


print(f'accuracy: {acc/len(df.path_img.unique())}')

100%|██████████| 904/904 [00:01<00:00, 871.00it/s]

accuracy: 0.9800884955752213


In [15]:
acc = 0
empty_count = 0
for path in tqdm(df.path_img.unique()):
    res = predict_df[predict_df.path_img == path]
    if len(res) == 0:
        empty_count += 1
        continue

    res['w'] = (res['x2'] - res['x1'])
    res['h'] = (res['y2'] - res['y1'])
    res['area'] = (res.w * res.h)

    temp = res.sort_values('area', ascending=False)
    cls_type = temp['name'].iloc[0]
    # print(cls_type)
    # break
    # temp = res.groupby('name').count()
    # temp = temp.sort_values('conf', ascending=False)
    # cls_type = None
    # if len(temp)>=2 and temp.conf.iloc[0] == temp.conf.iloc[1]:
    #     # если нет доминантного класса по количеству
    #     # самый уверенный бокс
    #     # res = predict_df[predict_df.path_img == path]
    #     res = res.sort_values('conf', ascending=False)
    #     cls_type = res['name'].iloc[0]
    # elif len(temp):
    #     cls_type = temp.index[0]
    if cls_type == df[df.path_img==path]['type_name'].iloc[0]:
        acc += 1

print(f'accuracy: {acc/len(df.path_img.unique())}')
print(f'Empty predict count = {empty_count}')

  0%|          | 0/904 [00:00<?, ?it/s]C:\Users\annza\AppData\Local\Temp\ipykernel_10384\478804981.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res['w'] = (res['x2'] - res['x1'])
C:\Users\annza\AppData\Local\Temp\ipykernel_10384\478804981.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res['h'] = (res['y2'] - res['y1'])
C:\Users\annza\AppData\Local\Temp\ipykernel_10384\478804981.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

accuracy: 0.9756637168141593
Empty predict count = 3


In [16]:
predict_df

,name,conf,x1,y1,x2,y2,path_img,true_label
0,klikun,0.985566,146,96,580,326,klikun\images\1428654680.jpg,klikun
1,klikun,0.973267,224,33,614,238,klikun\images\1429006927.jpg,klikun
2,klikun,0.962034,115,111,515,343,klikun\images\1429006927.jpg,klikun
3,klikun,0.973135,0,99,299,580,klikun\images\1429232757.jpg,klikun
4,klikun,0.983469,127,26,475,322,klikun\images\1429739095.jpg,klikun
...,...,...,...,...,...,...,...,...
1340,shipun,0.954823,114,214,529,356,shipun\images\img_972.jpg,shipun
1341,shipun,0.935426,0,131,338,258,shipun\images\img_972.jpg,shipun
1342,shipun,0.970300,454,66,640,220,shipun\images\img_978.jpg,shipun
1343,shipun,0.822528,226,145,606,375,shipun\images\img_978.jpg,shipun


In [17]:
predict_df.to_csv('predict_test.csv', index=False)